<a href="https://colab.research.google.com/github/nsk-ai/RAG-Bootcamp-2025/blob/main/Dense_Vector_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dense Vector Search

In [ ]:
pip install langchain

In [ ]:
pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

# Access the secret using userdata.get()
my_variable = userdata.get('GROQ_API_KEY')

# You can also set it as an environment variable for use with os.getenv()
os.environ['GROQ_API_KEY'] = my_variable

In [ ]:
pip install -qU langchain-community chromadb faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00

In [ ]:
import os
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 1. Initialize our Embedding Model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print("Embedding model initialized.")

/tmp/ipython-input-3485307554.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model initialized.


In [ ]:
# 2. Define our Sample Documents with varied content and metadata
docs = [
    Document(
        page_content="The new CEO, Sarah Jenkins, announced a company-wide focus on AI integration.",
        metadata={"source": "press-release", "year": 2024, "author": "CorpComms"}
    ),
    Document(
        page_content="Our quarterly report shows a 15% growth in the tech sector, largely driven by AI advancements.",
        metadata={"source": "financial-report", "year": 2024, "author": "FinanceDept"}
    ),
    Document(
        page_content="To reset your password, please visit the account portal and click 'Forgot Password'.",
        metadata={"source": "faq", "topic": "account-management"}
    ),
    Document(
        page_content="If you are unable to access your account, please follow the password recovery steps.",
        metadata={"source": "faq", "topic": "account-management"}
    ),
    Document(
        page_content="The arctic fox is known for its thick, white fur that provides camouflage in the snow.",
        metadata={"source": "nature-facts", "topic": "animals"}
    ),
    Document(
        page_content="A swift, reddish-brown canine leaps over a slumbering dog in the classic pangram.",
        metadata={"source": "literature", "topic": "animals"}
    )
]

In [ ]:
# 3. Create our Vector Store (using ChromaDB for this example)
# We'll use a persistent directory to store our DB.
CHROMA_DB_DIR = "./chroma_db_dense_search"
db = Chroma.from_documents(docs, embeddings, persist_directory=CHROMA_DB_DIR)
print(f"ChromaDB initialized and populated at {CHROMA_DB_DIR}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


ChromaDB initialized and populated at ./chroma_db_dense_search


### Example 1: Basic `similarity_search`

This is our baseline. Let's perform a standard semantic search and see how it handles synonyms and paraphrased questions.

In [ ]:
# Create a standard similarity retriever
retriever_similarity = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2} # We want the top 2 most similar documents
)

# --- Test Case 1: Paraphrased Query ---
query_paraphrase = "How can I regain access to my account?"
retrieved_docs_para = retriever_similarity.invoke(query_paraphrase)

print(f"\n--- Basic Similarity Search for: '{query_paraphrase}' ---")
for i, doc in enumerate(retrieved_docs_para):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Source='{doc.metadata.get('source')}', Topic='{doc.metadata.get('topic')}'\n")


--- Basic Similarity Search for: 'How can I regain access to my account?' ---
Doc 1: Content='If you are unable to access your account, please follow the password recovery steps.'
   Source='faq', Topic='account-management'

Doc 2: Content='To reset your password, please visit the account portal and click 'Forgot Password'.'
   Source='faq', Topic='account-management'



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# --- Test Case 2: Conceptual Query ---
query_conceptual = "What's the latest news on artificial intelligence in the company?"
retrieved_docs_concept = retriever_similarity.invoke(query_conceptual)

print(f"\n--- Basic Similarity Search for: '{query_conceptual}' ---")
for i, doc in enumerate(retrieved_docs_concept):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Source='{doc.metadata.get('source')}', Year='{doc.metadata.get('year')}'\n")


--- Basic Similarity Search for: 'What's the latest news on artificial intelligence in the company?' ---
Doc 1: Content='The new CEO, Sarah Jenkins, announced a company-wide focus on AI integration.'
   Source='press-release', Year='2024'

Doc 2: Content='Our quarterly report shows a 15% growth in the tech sector, largely driven by AI advancements.'
   Source='financial-report', Year='2024'



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### Example 2: Using `search_type="mmr"` for Diversity

Now, let's look at a scenario where a standard search might give us redundant results. Our two FAQ documents are very similar. Let's craft a query that retrieves them and see how MMR can give us a more diverse set of results.

In [ ]:
# Create an MMR retriever
mmr_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2, "fetch_k": 4} # Fetch 4, but return the best 2 diverse results
)

In [ ]:
query_mmr = "Tell me about account help and animals." # A broad query to test diversity

In [ ]:
# First, let's see what the standard similarity retriever does
retrieved_docs_sim = retriever_similarity.invoke(query_mmr)
print(f"\n--- Standard Similarity Search Results for: '{query_mmr}' ---")
for i, doc in enumerate(retrieved_docs_sim):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Topic='{doc.metadata.get('topic')}'\n")


--- Standard Similarity Search Results for: 'Tell me about account help and animals.' ---
Doc 1: Content='If you are unable to access your account, please follow the password recovery steps.'
   Topic='account-management'

Doc 2: Content='To reset your password, please visit the account portal and click 'Forgot Password'.'
   Topic='account-management'



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# Now, let's see what the MMR retriever does
retrieved_docs_mmr = mmr_retriever.invoke(query_mmr)
print(f"\n--- MMR Search Results for: '{query_mmr}' ---")
print("Note how MMR tries to pick one from each topic.")
for i, doc in enumerate(retrieved_docs_mmr):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Topic='{doc.metadata.get('topic')}'\n")


--- MMR Search Results for: 'Tell me about account help and animals.' ---
Note how MMR tries to pick one from each topic.
Doc 1: Content='If you are unable to access your account, please follow the password recovery steps.'
   Topic='account-management'

Doc 2: Content='A swift, reddish-brown canine leaps over a slumbering dog in the classic pangram.'
   Topic='animals'



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


> #### **Key Takeaway: When to use MMR**
>
> Use **`search_type="mmr"`** when you want to avoid getting multiple, very similar chunks of text in your results. It's perfect for when your source documents contain lots of overlapping or redundant information and you want to provide the LLM with a broader, more diverse context.

### Example 3: Dense Search with Metadata Filtering

This is where you can combine the best of both worlds: semantic meaning and precise filtering.


In [ ]:
# Create a retriever that ONLY searches within documents from the "faq" source
faq_retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1, "filter": {"source": "faq"}}
)

In [ ]:
query_filtered = "How do I get into my account?"
retrieved_docs_filtered = faq_retriever.invoke(query_filtered)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
print(f"\n--- Filtered Dense Search for: '{query_filtered}' (Source='faq') ---")
for i, doc in enumerate(retrieved_docs_filtered):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Source='{doc.metadata.get('source')}'\n")


--- Filtered Dense Search for: 'How do I get into my account?' (Source='faq') ---
Doc 1: Content='If you are unable to access your account, please follow the password recovery steps.'
   Source='faq'



In [ ]:
# Example with a numerical filter (year must be 2024)
# Note: ChromaDB supports operators like $eq, $ne, $gt, $gte, $lt, $lte
recent_report_retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2, "filter": {"year": {"$eq": 2024}}}
)

In [ ]:
query_numerical = "What are the latest developments?"
retrieved_num_filtered = recent_report_retriever.invoke(query_numerical)
print(f"\n--- Filtered Dense Search for: '{query_numerical}' (Year = 2024) ---")
for i, doc in enumerate(retrieved_num_filtered):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Source='{doc.metadata.get('source')}', Year='{doc.metadata.get('year')}'\n")


--- Filtered Dense Search for: 'What are the latest developments?' (Year = 2024) ---
Doc 1: Content='Our quarterly report shows a 15% growth in the tech sector, largely driven by AI advancements.'
   Source='financial-report', Year='2024'

Doc 2: Content='The new CEO, Sarah Jenkins, announced a company-wide focus on AI integration.'
   Source='press-release', Year='2024'



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
